# Cell 1 (Markdown)
# OpenAI Function Calling Example

This notebook demonstrates how to use OpenAI's function calling capability to create an interactive weather assistant. We'll go through the process step by step:

1. Setting up the tools (functions)
2. Making the initial model call
3. Executing the function based on model's decision
4. Processing the results
5. Getting the final response

In [7]:
import json
import os

import requests
from openai import OpenAI
from pydantic import BaseModel, Field

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [8]:
def get_weather(latitude, longitude):
    """This is a publically available API that returns the weather for a given location."""
    response = requests.get(
        f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m"
    )
    data = response.json()
    return data["current"]

In [9]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get current temperature for provided coordinates in celsius.",
            "parameters": {
                "type": "object",
                "properties": {
                    "latitude": {"type": "number"},
                    "longitude": {"type": "number"},
                },
                "required": ["latitude", "longitude"],
                "additionalProperties": False,
            },
            "strict": True,
        },
    }
]

system_prompt = "You are a helpful weather assistant."

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "What's the weather like in Paris today?"},
]

In [4]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
)

print("Model's response:")
completion.model_dump()

Model's response:


{'id': 'chatcmpl-Awc7Bcaj2y9LT97XyOs85Do3UXEpn',
 'choices': [{'finish_reason': 'tool_calls',
   'index': 0,
   'logprobs': None,
   'message': {'content': None,
    'refusal': None,
    'role': 'assistant',
    'audio': None,
    'function_call': None,
    'tool_calls': [{'id': 'call_mCgzjk068dxQUDtw7dizM9cO',
      'function': {'arguments': '{"latitude":48.8566,"longitude":2.3522}',
       'name': 'get_weather'},
      'type': 'function'}]}}],
 'created': 1738532901,
 'model': 'gpt-4o-mini-2024-07-18',
 'object': 'chat.completion',
 'service_tier': 'default',
 'system_fingerprint': 'fp_72ed7ab54c',
 'usage': {'completion_tokens': 25,
  'prompt_tokens': 66,
  'total_tokens': 91,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens': 0,
   'rejected_prediction_tokens': 0},
  'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}}

In [5]:
def call_function(name, args):
    if name == "get_weather":
        return get_weather(**args)

# Process tool calls and append results to messages
for tool_call in completion.choices[0].message.tool_calls:
    name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)
    messages.append(completion.choices[0].message)
    
    result = call_function(name, args)
    messages.append(
        {"role": "tool", "tool_call_id": tool_call.id, "content": json.dumps(result)}
    )

print("Function results:")
print(json.dumps(result, indent=2))

Function results:
{
  "time": "2025-02-02T21:45",
  "interval": 900,
  "temperature_2m": 1.7,
  "wind_speed_10m": 3.0
}


In [6]:
class WeatherResponse(BaseModel):
    temperature: float = Field(
        description="The current temperature in celsius for the given location."
    )
    response: str = Field(
        description="A natural language response to the user's question."
    )

completion_2 = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
    response_format=WeatherResponse,
)

final_response = completion_2.choices[0].message.parsed
print(f"Temperature: {final_response.temperature}°C")
print(f"Response: {final_response.response}")

Temperature: 1.7°C
Response: The current temperature in Paris is 1.7°C. It's a chilly day!
